In [23]:
# Use CC to detect waveforms similar to the twin events

In [24]:
## Import modules

from obspy import read, UTCDateTime, Stream
from utils_basic import ROOTDIR
from utils_cc import template_match_3c, associate_detections, get_timewin_from_template_name, TemplateEvent, TemplateMatches
from utils_preproc import preprocess_stream
from utils_snuffler import read_normal_markers
from os.path import join
from pandas import read_csv, concat

In [25]:
## Inputs
rootdir = ROOTDIR
tempname = "BeatB16-08-06-34"
timelen = 0.1 # Length of time window for CC in seconds

freqmin = 10 # Low corner frequency for bandpass filter
freqmax = 450 # High corner frequency for bandpass filter

mincc = 0.7 # Minimum CC coefficient for detection

numdet_min = 4 # Minimum number of detections to be considered a detected event

delta_max = 0.1 # Maximum difference in delta between detections to be considered the same event

In [26]:
## Read the picks of the master event
timewin_temp = get_timewin_from_template_name(tempname)

pkname = f"xpicks_{tempname}.dat"
inpath = join(rootdir, timewin_temp, pkname)
pickdf = read_normal_markers(inpath)

In [27]:
## Read and process the master event

### Read the hour containing the master event
stream_hour = Stream()
for stname in pickdf["station"]:
    stream_hour += read(join(rootdir, timewin_temp, f"*{stname}*SAC"))

### Preprocess the data
stream_hour = preprocess_stream(stream_hour, freqmin, freqmax)

### Cut the master event
stream_mast = Stream()

for i, row in pickdf.iterrows():
    stname = row["station"]
    starttime = UTCDateTime(row["time"])
    stream_mast += stream_hour.select(station=stname).slice(starttime, starttime+timelen)


In [28]:
## Read the time window list
inpath = join(rootdir, "time_windows.csv")
twindf = read_csv(inpath)

In [29]:
## Cross-correlate the master event with all the time windows
print(f"Processing {tempname}...")

detdfs = []
for stname in pickdf["station"]:
    stream_temp = stream_mast.select(station=stname)

    
    for timewin in twindf["name"]:
        print(f"Processing {stname}, {timewin}...")

        try:
            stream_targ = read(join(rootdir, timewin, f"*{stname}*SAC"))
        except:
            print(f"Cannot find data for {timewin}!")
            continue
        
        stream_targ = preprocess_stream(stream_targ, freqmin, freqmax)
        detdf = template_match_3c(stream_targ, stream_temp, tempname, mincc=mincc)

        if detdf is not None:
            detdfs.append(detdf)

detdf = concat(detdfs)
detdf.sort_values(by=["time"], ascending=True, inplace=True, ignore_index=True)
numdet = len(detdf)
dur = timelen

if freqmax is None:
    outfile = f"ccdetections_{tempname}_highpass{freqmin:.0f}hz_dur{dur:.2f}s_mincc{mincc:.1f}.csv"
else:
    outfile = f"ccdetections_{tempname}_bandpass{freqmin:.0f}-{freqmax:.0f}hz_dur{dur:.2f}s_mincc{mincc:.1f}.csv"

outpath = join(rootdir, timewin_temp, outfile)
detdf["time_shift"] = detdf["time_shift"].dt.total_seconds()
detdf.to_csv(outpath, sep=" ")
print(f"Results saved to {outpath}.")

Processing BeatB16-08-06-34...
Processing B04, 2020-01-10-11-00-00...
Cannot find data for 2020-01-10-11-00-00!
Processing B04, 2020-01-10-12-00-00...
Cannot find data for 2020-01-10-12-00-00!
Processing B04, 2020-01-10-13-00-00...


/opt/homebrew/Caskroom/miniforge/base/envs/obspy/lib/python3.11/site-packages/obspy/signal/cross_correlation.py:291: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norm, out=norm)


No detections found.
Processing B04, 2020-01-10-14-00-00...
No detections found.
Processing B04, 2020-01-10-15-00-00...
No detections found.
Processing B04, 2020-01-10-16-00-00...
No detections found.
Processing B04, 2020-01-10-17-00-00...
No detections found.
Processing B04, 2020-01-10-18-00-00...
No detections found.
Processing B04, 2020-01-10-19-00-00...
No detections found.
Processing B04, 2020-01-10-20-00-00...
No detections found.
Processing B04, 2020-01-10-21-00-00...
No detections found.
Processing B04, 2020-01-10-22-00-00...
No detections found.
Processing B04, 2020-01-10-23-00-00...
No detections found.
Processing B04, 2020-01-11-00-00-00...
No detections found.
Processing B04, 2020-01-11-01-00-00...
No detections found.
Processing B04, 2020-01-11-02-00-00...
No detections found.
Processing B04, 2020-01-11-03-00-00...
No detections found.
Processing B04, 2020-01-11-04-00-00...
No detections found.
Processing B04, 2020-01-11-05-00-00...
No detections found.
Processing B04, 202

/opt/homebrew/Caskroom/miniforge/base/envs/obspy/lib/python3.11/site-packages/obspy/signal/cross_correlation.py:291: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norm, out=norm)


No detections found.
Processing B04, 2020-01-11-20-00-00...
No detections found.
Processing B04, 2020-01-11-21-00-00...
1 detections found.
Processing B04, 2020-01-11-22-00-00...
3 detections found.
Processing B04, 2020-01-11-23-00-00...
2 detections found.
Processing B04, 2020-01-12-00-00-00...
No detections found.
Processing B04, 2020-01-12-01-00-00...
No detections found.
Processing B04, 2020-01-12-02-00-00...
No detections found.
Processing B04, 2020-01-12-03-00-00...
No detections found.
Processing B04, 2020-01-12-04-00-00...
2 detections found.
Processing B04, 2020-01-12-05-00-00...
No detections found.
Processing B04, 2020-01-12-06-00-00...
13 detections found.
Processing B04, 2020-01-12-07-00-00...
28 detections found.
Processing B04, 2020-01-12-08-00-00...
48 detections found.
Processing B04, 2020-01-12-09-00-00...
72 detections found.
Processing B04, 2020-01-12-10-00-00...
74 detections found.
Processing B04, 2020-01-12-11-00-00...
92 detections found.
Processing B04, 2020-01

...

2 detections found.
Processing B03, 2020-01-29-07-00-00...
No detections found.
Processing B03, 2020-01-29-08-00-00...
10 detections found.
Processing B03, 2020-01-29-09-00-00...
No detections found.
Processing B03, 2020-01-29-10-00-00...
No detections found.
Processing B03, 2020-01-29-11-00-00...
No detections found.
Processing B03, 2020-01-29-12-00-00...
No detections found.
Processing B03, 2020-01-29-13-00-00...
No detections found.
Processing B03, 2020-01-29-14-00-00...
No detections found.
Processing B03, 2020-01-29-15-00-00...
No detections found.
Processing B03, 2020-01-29-16-00-00...
No detections found.
Processing B03, 2020-01-29-17-00-00...
No detections found.
Processing B03, 2020-01-29-18-00-00...
No detections found.
Processing B03, 2020-01-29-19-00-00...
No detections found.
Processing B03, 2020-01-29-20-00-00...
No detections found.
Processing B03, 2020-01-29-21-00-00...
No detections found.
Processing B03, 2020-01-29-22-00-00...
No detections found.
Processing B03, 2020

/opt/homebrew/Caskroom/miniforge/base/envs/obspy/lib/python3.11/site-packages/obspy/signal/cross_correlation.py:291: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norm, out=norm)


No detections found.
Processing B03, 2020-01-31-12-00-00...
No detections found.
Processing B03, 2020-01-31-13-00-00...
No detections found.
Processing B03, 2020-01-31-14-00-00...
No detections found.
Processing B03, 2020-01-31-15-00-00...
No detections found.
Processing B03, 2020-01-31-16-00-00...
No detections found.
Processing B03, 2020-01-31-17-00-00...
No detections found.
Processing B03, 2020-01-31-18-00-00...
No detections found.
Processing B03, 2020-01-31-19-00-00...
No detections found.
Processing B03, 2020-01-31-20-00-00...
No detections found.
Processing B03, 2020-01-31-21-00-00...
No detections found.
Processing B03, 2020-01-31-22-00-00...
No detections found.
Processing B03, 2020-01-31-23-00-00...
No detections found.
Processing B03, 2020-02-01-00-00-00...
No detections found.
Processing B03, 2020-02-01-01-00-00...
No detections found.
Processing B03, 2020-02-01-02-00-00...
No detections found.
Processing B03, 2020-02-01-03-00-00...
No detections found.
Processing B03, 202

/opt/homebrew/Caskroom/miniforge/base/envs/obspy/lib/python3.11/site-packages/obspy/signal/cross_correlation.py:291: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norm, out=norm)


2 detections found.
Processing B03, 2020-02-02-13-00-00...
No detections found.
Processing B03, 2020-02-02-14-00-00...
No detections found.
Processing B03, 2020-02-02-15-00-00...
No detections found.
Processing B03, 2020-02-02-16-00-00...
No detections found.
Processing B03, 2020-02-02-17-00-00...
No detections found.
Processing B03, 2020-02-02-18-00-00...
No detections found.
Processing B03, 2020-02-02-19-00-00...
No detections found.
Processing B03, 2020-02-02-20-00-00...
No detections found.
Processing B03, 2020-02-02-21-00-00...
No detections found.
Processing B03, 2020-02-02-22-00-00...
No detections found.
Results saved to /Volumes/OmanData/geophones_no_prefilt/data/2020-01-16-08-00-00/ccdetections_BeatB16-08-06-34_bandpass10-450hz_dur0.10s_mincc0.7.csv.


In [30]:
## Associate the detected events
# import importlib
# import cc_utils
# importlib.reload(cc_utils)

#matches = cc_utils.associate_detections(detdf, numdet_min=numdet_min, delta_max=delta_max)
matches = associate_detections(detdf, numdet_min=numdet_min, delta_max=delta_max)
    

There are in total 9655 matches.


In [31]:
# Save the detected events
# import importlib
# import cc_utils
# importlib.reload(cc_utils)

stnames = pickdf["station"].tolist()
starttimes = pickdf["time"].tolist()
template = TemplateEvent(tempname, dur, stnames, starttimes)
tempmatch = TemplateMatches(freqmin, freqmax, mincc, numdet_min, template, matches)
# template = cc_utils.TemplateEvent(tempname, dur, stnames, starttimes)
# tempmatch = cc_utils.TemplateMatches(freqmin, freqmax, mincc, numdet_min, template, matches)

if freqmax is None:
    outfile = f"ccmatches_{tempname}_highpass{freqmin:.0f}hz_dur{dur:.2f}s_mincc{mincc:.1f}_mindet{numdet_min:d}.dat"
else:
    outfile = f"ccmatches_{tempname}_bandpass{freqmin:.0f}-{freqmax:.0f}hz_dur{dur:.2f}s_mincc{mincc:.1f}_mindet{numdet_min:d}.dat"

outpath = join(rootdir, timewin_temp, outfile)

tempmatch.write_to_file(outpath)

Template and match information is saved to /Volumes/OmanData/geophones_no_prefilt/data/2020-01-16-08-00-00/ccmatches_BeatB16-08-06-34_bandpass10-450hz_dur0.10s_mincc0.7_mindet4.dat
